# Multi-Class Classification with Perceptron

Lab Assignment from [AI for Beginners Curriculum](https://github.com/microsoft/ai-for-beginners).

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import gzip
import pickle
import os
import random
import time

In [2]:
with gzip.open('../data/mnist.pkl.gz', 'rb') as mnist_pickle:
    MNIST = pickle._Unpickler(mnist_pickle, encoding='latin1').load()

In [3]:
n = 42000 #Total # of examples in dataset

train_x, test_x = np.split(MNIST['Train']['Features'], [n*8//10]) #// is integer division, likely cuz can't pass float to np.split
train_labels, test_labels = np.split(MNIST['Train']['Labels'], [n*8//10])

You can use the following perceptron training code from the lecture:

In [4]:
def train(positive_examples, negative_examples, num_iterations = 100, report=False):
    num_dims = positive_examples.shape[1]
    weights = np.zeros((num_dims,1)) # initialize weights
    
    pos_count = positive_examples.shape[0]
    neg_count = negative_examples.shape[0]
    
    report_frequency = 10
    
    for i in range(num_iterations):
        pos = random.choice(positive_examples)
        neg = random.choice(negative_examples)

        z = np.dot(pos, weights)   
        if z < 0:
            weights = weights + pos.reshape(weights.shape)

        z  = np.dot(neg, weights)
        if z >= 0:
            weights = weights - neg.reshape(weights.shape)

        if report:
            if i % report_frequency == 0:             
                pos_out = np.dot(positive_examples, weights)
                neg_out = np.dot(negative_examples, weights)        
                pos_correct = (pos_out >= 0).sum() / float(pos_count)
                neg_correct = (neg_out < 0).sum() / float(neg_count)
                print("Iteration={}, pos correct={}, neg correct={}".format(i,pos_correct,neg_correct))

    return weights

In [5]:
def accuracy(weights, test_x, test_labels, pos):
    res = np.dot(test_x, weights)
    correct_pos = sum([1 for i, _ in enumerate(res) if test_labels[i] == pos and res[i] >= 0])
    correct_neg = sum([1 for i, _ in enumerate(res) if test_labels[i] != pos and res[i] < 0])

    return (correct_pos+correct_neg)/float(len(test_labels))

In [6]:
def set_mnist_pos_neg(positive_label):
    positive_indices = [i for i, j in enumerate(train_labels) 
                          if j == positive_label]
    negative_indices = [i for i, j in enumerate(train_labels) 
                          if j != positive_label]

    positive_images = MNIST['Train']['Features'][positive_indices]
    negative_images = MNIST['Train']['Features'][negative_indices]

    return positive_images, negative_images

Now you need to:
1. Create 10 *one-vs-all* datasets for all digits
1. Train 10 perceptrons
1. Define `classify` function to perform digit classification
1. Measure the accuracy of classification and print *confusion matrix*
1. [Optional] Create improved `classify` function that performs the classification using one matrix multiplication.

In [7]:
all_wts = []
for i in range(0, 10):
    pos, neg = set_mnist_pos_neg(i)
    wt = train(pos, neg)
    all_wts.append(wt)
    print(f"{i} Accuracy: {accuracy(wt, test_x, test_labels, pos=i)}")


0 Accuracy: 0.9307142857142857
1 Accuracy: 0.9251190476190476
2 Accuracy: 0.8917857142857143
3 Accuracy: 0.9330952380952381
4 Accuracy: 0.9486904761904762
5 Accuracy: 0.8478571428571429
6 Accuracy: 0.6378571428571429
7 Accuracy: 0.8285714285714286
8 Accuracy: 0.8875
9 Accuracy: 0.7773809523809524


In [8]:
### Randomly selects a sample
### multiplies weights by the sample features
### The highest product is assumed to be the most confident prediction
### Index of highest prediction is the number predicted

def classify(weights):
    random_sample = random.randint(0, len(test_x))
    sample_features = test_x[random_sample]
    sample_label = test_labels[random_sample]

    print(f'Sample Number = {sample_label}')

    predictions = [np.dot(weights[i].reshape(sample_features.shape), sample_features) for i in range(0, 10)]
    max_prediction = max(predictions)
    predicted_label = predictions.index(max_prediction)
    correct =  predicted_label == sample_label
    if correct:
        print(f'Guessed Number = {predicted_label}')
    else:
        print(f'Hmmmm, incorrect. Guessed Number = {predicted_label}')


In [10]:
classify(all_wts)

Sample Number = 3
Guessed Number = 3
